# The Kern River (CA) floods in spring 2023
![Kern River in the South Sierra Wilderness](images/kern_river_2023.jpg)

**photo credit: Johanna Kinsler**

In 2023 the Kern River had high spring discharge due to record winter 2022/2023 snowfall.

In [1]:
# !pip3 install folium
# !pip3 install requests
# !pip3 install hvplot
# !pip3 install pandas



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import folium 
from io import BytesIO
import requests
import subprocess
import hvplot.pandas
import pandas as pd

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


## STEP 2: SITE MAP AND DESCRIPTION
<!-- ![](kern_river.mov) -->


### All throughout the Sierra drainage rivers flooded and reservoirs refilled
This analysis focuses on the Kern River, near Onyx CA.

### Site Description
  * The data was collected at South Fork Kern River in Onyx, CA, site code 11189500
  * The region is characterized with rugged mountains and a high desert climate. The majority of precipitation feeding the river arrives as snowfall between December and March.
  * The agency that manages and collects this data is USGS
  * You can access this usgs tat through their [national water dashboard](https://dashboard.waterdata.usgs.gov/app/nwd/en/?aoi=default)
 

### Site Map: The Kern River near Onyx, CA

In [6]:
sg_lat = 35.7374516
sg_lon = -118.173689

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Thingy").add_to(m)

# Display the map
m

&#9998; USGS streamflow URL: *url here*

## Data Exploration

In [7]:
streamflow_url = ("https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format"
                  "=rdb&site_no=11189500&legacy=&referred_module=sw&period"
                  "=&begin_date=1911-09-12&end_date=2023-09-22"
    )

# streamflow_url

'https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=11189500&legacy=&referred_module=sw&period=&begin_date=1911-09-12&end_date=2023-09-22'

In [8]:
# Send an HTTP GET request to the URL
nwis_response = requests.get(streamflow_url)

# Check if the request was successful (status code 200)
if nwis_response.status_code == 200:
    # Print the content of the nwis_response (HTML, JSON, etc.)
    print(nwis_response.text)
else:
    print(f"Request failed with status code {nwis_response.status_code}")

# ---------------------------------- WARNING ----------------------------------------
# Some of the data that you have obtained from this U.S. Geological Survey database
# may not have received Director's approval. Any such data values are qualified
# as provisional and are subject to revision. Provisional data are released on the
# condition that neither the USGS nor the United States Government may be held liable
# for any damages resulting from its use.
#
# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement
#
# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output
# Automated-retrieval info: https://help.waterdata.usgs.gov/faq/automated-retrievals
#
# Contact:   gs-w_waterdata_support@usgs.gov
# retrieved: 2023-09-23 17:12:18 EDT       (vaww01)
#
# Data for the following 1 site(s) are contained in this file
#    USGS 11189500 SF KERN R NR ONYX CA
# -------------------------------------------------------------------

In [25]:
# Print the top of the data
for i, line in enumerate(nwis_response.content.splitlines()[:10]):
    print(i, line)

0 b'# ---------------------------------- WARNING ----------------------------------------'
1 b'# Some of the data that you have obtained from this U.S. Geological Survey database'
2 b"# may not have received Director's approval. Any such data values are qualified"
3 b'# as provisional and are subject to revision. Provisional data are released on the'
4 b'# condition that neither the USGS nor the United States Government may be held liable'
5 b'# for any damages resulting from its use.'
6 b'#'
7 b'# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement'
8 b'#'
9 b'# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output'


In [26]:
# Take a look at the data. What got downloaded?
for i, line in enumerate(nwis_response.content.splitlines()[:35]):
    # Skip commented lines
    if not line.startswith(b'#'):
        print(i, line)

27 b'agency_cd\tsite_no\tdatetime\t43245_00060_00003\t43245_00060_00003_cd'
28 b'5s\t15s\t20d\t14n\t10s'
29 b'USGS\t06610000\t1928-10-01\t20300\tA'
30 b'USGS\t06610000\t1928-10-02\t19800\tA'
31 b'USGS\t06610000\t1928-10-03\t19600\tA'
32 b'USGS\t06610000\t1928-10-04\t19300\tA'
33 b'USGS\t06610000\t1928-10-05\t19000\tA'
34 b'USGS\t06610000\t1928-10-06\t18000\tA'


In [19]:
# Import the data using pandas

discharge_df = pd.read_csv(
    BytesIO(nwis_response.content),
    comment='#',
    delimiter='\t', 
    skiprows=[i for i in range(31)],
    names=['agency_code', 'site_number', 'date', 'stream_discharge_cfs', 'site_code'],
    index_col='date',
    parse_dates=['date'],
)

discharge_df

/var/folders/m2/hyd53wyn2ks8hlxdc01nr8m00000gn/T/ipykernel_14849/487946314.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  discharge_df = pd.read_csv(


,agency_code,site_number,stream_discharge_cfs,site_code
date,,,,
20d,5s,15s,14n,10s
1911-09-12,USGS,11189500,34.0,A
1911-09-13,USGS,11189500,34.0,A
1911-09-14,USGS,11189500,32.0,A
1911-09-15,USGS,11189500,31.0,A
...,...,...,...,...
2023-09-18,USGS,11189500,74.1,P
2023-09-19,USGS,11189500,72.7,P
2023-09-20,USGS,11189500,71.4,P


### Data Schema

In [20]:
discharge_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40920 entries, 20d to 2023-09-22
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   agency_code           40920 non-null  object
 1   site_number           40920 non-null  object
 2   stream_discharge_cfs  35405 non-null  object
 3   site_code             35405 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


### Data coverage & date range

In [21]:
discharge_df.index

Index(['20d', '1911-09-12', '1911-09-13', '1911-09-14', '1911-09-15',
       '1911-09-16', '1911-09-17', '1911-09-18', '1911-09-19', '1911-09-20',
       ...
       '2023-09-13', '2023-09-14', '2023-09-15', '2023-09-16', '2023-09-17',
       '2023-09-18', '2023-09-19', '2023-09-20', '2023-09-21', '2023-09-22'],
      dtype='object', name='date', length=40920)

### Descriptive Statistics

In [22]:
discharge_df.describe()

,agency_code,site_number,stream_discharge_cfs,site_code
count,40920,40920,35405,35405
unique,2,2,2679,6
top,USGS,11189500,24.0,A
freq,40919,40919,439,34647


## Data Visualizaton
I hiked the South Fork PCT, Kern river in May 2023. Flood waters were especially high at that time. The plots below highlight spring 2023 discharge in comparison to annual maxiumums from the last 110 years.

In [30]:
spring_2023_df = discharge_df['2023-02-01':'2023-06-30']


spring_2023_df

,agency_code,site_number,stream_discharge_cfs,site_code
date,,,,
2023-02-01,USGS,11189500,55.7,P
2023-02-02,USGS,11189500,55.6,P
2023-02-03,USGS,11189500,55.1,P
2023-02-04,USGS,11189500,54.7,P
2023-02-05,USGS,11189500,61.4,P
...,...,...,...,...
2023-06-26,USGS,11189500,705,P
2023-06-27,USGS,11189500,664,P
2023-06-28,USGS,11189500,599,P


In [28]:
discharge_plot_feb_april = feb_april.hvplot(
    y='stream_discharge_cfs'
)

discharge_plot_feb_april

NameError: name 'feb_april' is not defined

You should be able to see the flood in your data going up above 188,000 cfs at its peak in March 2019, as compared to the following year where the peak flow was 71,100 cfs. But how unusual is that really?

Let's start by plotting ALL the data. Then you can (optionally) use a return period **statistic** to quantify how unusual it was.

&#128187; In the cell below, plot the entire time series of streamflow data, without any parameters.

In [107]:
discharge_plot_all = discharge_df.hvplot(
    y='stream_discharge_cfs'
)

discharge_plot_all

/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [date]   (stream_discharge_cfs)

This plot looks a little fuzzy because it is trying to fit too many data points in a small area. One way to improve this is by **resampling** the data to **annual maxima**. That way we still get the same peak streamflows, but the computer will be able to plot all the values without overlapping.

> **Resampling** means changing the time interval between time series observations - in this case from daily to annual.

&#128214; Read about [different ways to resample time series data in your textbook](https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/date-time-types-in-pandas-python/resample-time-series-data-pandas-python/)

&#128214; You can use a [list of **offset aliases**](https://pandas.pydata.org/docs/dev/user_guide/timeseries.html#timeseries-offset-aliases) to look up how to specify the final dates. This list is pretty hard to find - you might want to bookmark it.

&#128187; In the cell below, select the streamflow column, and then resample it to get an annual maximum.

> GOTCHA ALERT - the test below is looking for a pandas `DataFrame`, but when we select a single column we get a pandas `Series` (a `DataFrame` is a collection of `Series` where each column is one `Series`.) To get a `DataFrame` with a single column, use the syntax below with **two** square brackets:

```python
dataframe[['column_name']]
```

In [114]:
discharge_annual_max_df = discharge_df[['stream_discharge_cfs']].resample(
    'AS').max()

discharge_annual_df

,stream_discharge_cfs
date,
1928-01-01,26200
1929-01-01,198000
1930-01-01,84400
1931-01-01,51700
1932-01-01,129000
...,...
2018-01-01,113000
2019-01-01,188000
2020-01-01,73700


In [115]:
ans_resample = _
resample_points = 0

# Answer should be a DataFrame
if isinstance(ans_resample, pd.DataFrame):
    print("\u2705 Great, you created a pandas DataFrame above")
    resample_points += 1
else:
    print("\u274C Oops - the cell above should have a DataFrame output.")

# Answer should include 95 years of data
if len(ans_resample)==95:
    print("\u2705 Your DataFrame has the right number of years")
    resample_points += 2
else:
    print("\u274C Oops - did you resample your DataFrame to annual?")

# The mean of the streamflow Series should be 85074
if round(int(ans_resample.mean().iloc[0]), 0)==85074:
    print("\u2705 Your annual max streamflow DataFrame has the expected "
          "values in it, good job!")
    resample_points += 2
else:
    print("\u274C Your annual max streamflow DataFrame does not have the "
          "expected values in it.")

print("\u27A1 You received {} out of 5 points for subsetting the "
      "streamflow data.".format(
    resample_points))
resample_points

✅ Great, you created a pandas DataFrame above
✅ Your DataFrame has the right number of years
✅ Your annual max streamflow DataFrame has the expected values in it, good job!
➡ You received 5 out of 5 points for subsetting the streamflow data.


5

&#128187; Plot your resampled data.

In [118]:
discharge_annual_max_df.hvplot()

/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [date]   (stream_discharge_cfs)

In the cell below, write a headline and 2-3 sentence description of your plot. What do you estimate the return period was for the flood in 2019?

### Maximum streamflow in Omaha has intermittent high peaks.
Maximum stream flow values in Omaha, Nebraska were stable between 1960 and 2010. However there were execptional peaks in the 1950s, 2011, and 2019.

> It turns out that the USGS also has **peak streamflow** values for years going back to the 1800s in Omaha. For your own analysis, one option would be to analyse the peak streamflow values instead, and compare them to the ones you computed in the years you have data. **GOTCHA ALERT** - the peak streamflow values from the USGS will be higher, because the daily data is a daily **average** rather than an instantaneous **maximum** like the peak flow.

## STEP 6 (OPTIONAL): CALCULATE THE FLOOD RETURN PERIOD 

&#127798; EXTRA CHALLENGE In the cell below, calculate the exceedence probability and return period for each year of the **annual** data, and add them as columns to your DataFrame. To pass the tests, subset data after 1953, when there was a regime change evident on the river. What do you think caused that?.

> HINT: pandas columns have a `rank` method, which you can use. BUT -- you will need to use the `ascending=False` parameter, since higher rank should be lower exceedence probability 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
ans_return = _
return_points = 0

# Answer should be a DataFrame
if isinstance(ans_return, pd.DataFrame):
    print("\u2705 Great, you created a pandas dataframe above")
    return_points += 1
else:
    print("\u274C Oops - the cell above should have a DataFrame output.")

# Answer should include 70 years of data
if len(ans_return)==70:
    print("\u2705 Your DataFrame has the right number of days")
    return_points += 2
elif len(ans_return) > 70:
    print("\u274C Your DataFrame has too many years.")
else:
    print("\u274C Your DataFrame has too few years.")

# The value "hash" should be 171186.0
if round(ans_return.mean().product(), 0)==171186.0:
    print("\u2705 Your streamflow DataFrame has the expected values "
          "in it, good job!")
    return_points += 2
else:
    print("\u274C Your streamflow DataFrame does not have the "
          "expected values in it.")
    
    # Subtract one point for any PEP-8 errors
tmp_path = "tmp.py"
with open(tmp_path, "w") as tmp_file:
    tmp_file.write(In[-2])
ignore_flake8 = 'W292,F401,E302,F821,W503'
flake8_out = subprocess.run(
    ['flake8', 
     '--ignore', ignore_flake8, 
     '--import-order-style', 'edited',
     '--count', 
     tmp_path],
    stdout=subprocess.PIPE,
).stdout.decode("ascii")
print(flake8_out)
return_points -= int(flake8_out.splitlines()[-1])

print("\u27A1 You received {} out of 5 extra credit points for calculating "
      "return periods.".format(return_points))
return_points